In [202]:
!pip install pypdf

In [203]:
import os

os.chdir("../")

In [204]:
%pwd

'c:\\'

In [210]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.document_loaders import TextLoader

In [206]:
def Extract_Data(pdf_paths):
    loader = DirectoryLoader(
        pdf_paths,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

# def Extract_Data_txt(txt_paths):
#     loader = DirectoryLoader(
#         txt_paths,
#         glob="*.txt",
#         loader_cls=TextLoader  # ✔ correct loader for text files
#     )
#     documents = loader.load()
#     return documents

In [211]:
extracted_data=Extract_Data("C:\\Linkdin projects\\Medical - system\\Medical-system\\data\\Emergency")

# extracted_data = Extract_Data("C:\\Linkdin projects\\Medical - system\\Medical-system\\data")
# txt_extracted = Extract_Data_txt('C:\\Linkdin projects\\Medical - system\\Medical-system\\data\\Author')

PdfReadError("Invalid Elementary Object starting with b'P' @21741325: b'6 0 obj<</Universal PDF(The process that creates this PDF constitutes a trade se'")


In [212]:
extracted_data
# txt_extracted

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': 'C:\\Linkdin projects\\Medical - system\\Medical-system\\data\\Emergency\\Tintinalli’s Emergency Medicine Manual ( PDFDrive ).pdf', 'total_pages': 1089, 'page': 0, 'page_label': 'Cover'}, page_content=''),
 Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': 'C:\\Linkdin projects\\Medical - system\\Medical-system\\data\\Emergency\\Tintinalli’s Emergency Medicine Manual ( PDFDrive ).pdf', 'total_pages': 1089, 'page': 1, 'page_label': 'i'}, page_content='Tintinalli’s  \nEmergency Medicine Manual\n00_Cydulka_FM.indd   1 03/05/17   2:16 pm'),
 Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': 'C:\\Linkdin projects\\Medical - system\\Medical-system\\data\\Emergency\\Tintinalli’s Emergency Medicine Manual ( PDFDrive ).pdf', 'total_pages': 1089, 'page': 2, 'page_label': 'ii'}, page_content='Notice\nMedicine is an ever-changing

In [225]:
len(extracted_data)

1089

In [226]:
from typing import List
from langchain.schema import Document

def filter_to_miniDocs(docs: List[Document]) -> List[Document]:
    """
     Givin you a list of document object, return a new list of documents objects
     containing only 'source' in metadata and the orignal page_content  
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content, metdata = {"source" : src}
            )
        )
    return minimal_docs


In [227]:
minimal_docs = filter_to_miniDocs(extracted_data)
# minimal_docs #it should return sorce and page_content
# abdelrhman_hassan_data = filter_to_miniDocs(txt_extracted)

In [228]:
# chunking
def txt_split(docs):  
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        chunk_overlap=20,
        length_function = len
    )
    chunks = text_splitter.split_documents(docs)
    return chunks

In [229]:
chunks = txt_split(minimal_docs)
print("Numebr of chunks: ", len(chunks))
# chunks_txt = txt_split(abdelrhman_hassan_data)
# print("Numebr of chunks: ", len(chunks_txt))
# chunks_txt

Numebr of chunks:  6129
Numebr of chunks:  11


[Document(metadata={}, page_content='AI and Machine Learning Experience:\n- 4 years in machine learning and deep learning\n- Worked with CNN, RNN, LSTM, GRU, and Transformers\n- Experience building custom models from scratch\n- Experience training Seq2Seq models\n- Implemented violence detection with video models\n- Implemented seismic detection for NASA Space Apps\n- Strong in feature extraction, embeddings, and classification workflows'),
 Document(metadata={}, page_content='Seismic Detection Program | Built an AI model to identify seismic activity within noisy datasets \nusing realtime signal processing | NASA Space Apps Challenge  \nâ€¢ ASL Action Detection | Developed real-time Arabic Sign Language recognition using MediaPipe and \nTransformers for gesture classification  \nâ€¢ Lip Reading Model | Designed an RNN-based system to predict spoken words from silent videos to \nsupport accessibility applications'),
 Document(metadata={}, page_content='â€¢ Violence Detection | Created a

In [230]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():

    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embeddings = download_embeddings()


In [231]:
from dotenv import load_dotenv
import os
dotenv_path = r"C:\Linkdin projects\Medical - system\Medical-system\.env"  # Use raw string for Windows paths
load_dotenv(dotenv_path=dotenv_path)

True

Set API keys for OpenAI and Pinecone so libraries can access them securely

In [232]:
PINECONE_API_KEY  = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY  = os.getenv("OPENAI_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

initializes the Pinecone client so you can start working with your indexes.


In [233]:
from pinecone import Pinecone
Pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=Pinecone_api_key)

In [234]:
from pinecone import ServerlessSpec 

# index_name = "abderlhmans-medical-chatbot" old
index_name = "medical-chatbot-abdelrhman"  # new

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

for existing data


In [235]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

For uploading data


In [236]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=chunks,
    embedding=embeddings,
    index_name=index_name
)

In [134]:
dswitch = Document(
    page_content="Abdelrhman Hassan is a highly talented AI Engineer with strong expertise in machine learning, computer vision, LLMOps, and advanced neural network architectures.",
     metadata = {"source" : "Linkdin"},
)

To Add Any Document Into Data We Use


In [135]:
docsearch.add_documents(documents=[dswitch])

['c7a40dbe-5452-41f8-b1ad-be318912f3fe']

In [237]:
retriever = docsearch.as_retriever(search_type = "similarity", search_kwargs={"k" : 3}) # top 3 k

In [240]:
retrieved_docs = retriever.invoke("Hi")
retrieved_docs

[Document(id='d626b35b-6cbf-4cbc-831a-4a9d7d0e1347', metadata={}, page_content='Beaudreau.\n152_Cydulka_Ch152.indd   844 22/03/17   9:33 AM'),
 Document(id='4dea8603-3117-4804-93d2-832cc87de91b', metadata={}, page_content='09_Cydulka_Ch09.indd   83 15/03/17   10:49 AM'),
 Document(id='f42452cc-4d69-49e1-8b61-7cc102cc59fe', metadata={}, page_content='167_Cydulka_Ch167.indd   919 22/03/17   11:56 AM')]

In [138]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-4o-mini")

In [183]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [245]:
# from langchain.prompts import ChatPromptTemplate

system_prompt = (
    "You are an Medical assistant for question-answering tasks. "
    "You are a knowledgeable assistant. Answer questions using ONLY the information below."
    "You are a knowledgeable medical assistant. "
    "Answer questions **using only the retrieved context** provided below. "
    "If the answer is not in the context, say 'I don't know.' "
    "Keep your answer concise and under three sentences or more but it must be effective.\n\n"
    "Context:\n{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [242]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [243]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response['answer'])


Acromegaly is a disorder in which the abnormal release of a particular chemical from the pituitary gland in the brain causes increased growth in bone and soft tissue, as well as a variety of other disturbances throughout the body. Gigantism is the condition when this abnormality occurs before bone growth stops, leading to unusual height. When the abnormality occurs after bone growth has stopped, it is called acromegaly.


In [246]:
response = rag_chain.invoke({"input": "hi"})
print(response['answer'])

I don't know.


In [199]:
response = rag_chain.invoke({"input": "i have a mid-abdominal cramps what is this"})
print(response['answer'])

I don't know.
